# Global WCS Hack Notebook


This notebook hacks on the default ImvizExample.ipynb notebook to experiment with the feasibility of a global WCS frame to avoid the distortion issue discussed in the 2023-06-12 Hack Session

Import modules needed for this notebook.

In [ ]:
import warnings
import tempfile

import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.table import Table
from astroquery.mast import Observations
from photutils.aperture import CircularAperture, SkyCircularAperture
from regions import PixCoord, CirclePixelRegion, CircleSkyRegion

from jdaviz import Imviz

%matplotlib inline

### Preview of the dataset:

The following dataset of the HorseHead Nebula plays nicely with our rotation code. Here's what the data looks just loaded natively into Imviz (I show this here to demonstrate the difference in the nuel rotation

In [ ]:
imviz_preview = Imviz()
from astropy.utils.data import get_pkg_data_filename
data_path = get_pkg_data_filename('tutorials/FITS-images/HorseHead.fits')
imviz_preview.load_data(data_path)
imviz_preview.link_data(link_type='wcs')
imviz_preview.show()

### Initial Setup

By default, this proposal would generate a NUEL-oriented WCS-only layer FIRST, before loading the actual data supplied. This will set the WCS-only layer as the reference data by default, effectively setting the sky as reference. 

In [ ]:
def init_imviz_nuel(data_to_load):
    imviz_prep = Imviz()
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        imviz_prep.load_data(data_to_load, data_label="temp_data")
    imviz_prep.plugins['Canvas Rotation'].set_north_up_east_left()
    nuel_angle = imviz_prep.plugins['Canvas Rotation'].angle
    print(nuel_angle)


    from astropy import units as u
    from jdaviz.configs.imviz.wcs_utils import _get_rotated_nddata_from_label
    WCS_reference_frame = _get_rotated_nddata_from_label(imviz_prep.app, "temp_data[PRIMARY,1]", nuel_angle*u.deg)
    WCS_reference_frame_45 = _get_rotated_nddata_from_label(imviz_prep.app, "temp_data[PRIMARY,1]", (nuel_angle+45)*u.deg)
    
    
    imviz = Imviz()
    imviz.load_data(WCS_reference_frame, data_label="nuel")
    imviz.load_data(WCS_reference_frame_45, data_label="nuel45")
    imviz.load_data(data_path)
    imviz.link_data(link_type='wcs')
    
    # reset zoom:
    viewer = imviz.app.get_viewer('imviz-0')
    viewer.shape = imviz.app.data_collection[-1]["PRIMARY,1"].shape
    viewer.state._set_axes_aspect_ratio(1)
    viewer.state.reset_limits()
    return imviz

imviz = init_imviz_nuel(data_path)
imviz.show()

In [ ]:
nuel = imviz.app.data_collection[0]
imviz.app.data_collection.remove(nuel)
imviz.app.data_collection